# Importações de dados brutos

In [115]:
import os
import joblib

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from scipy import stats
from scipy.stats import mannwhitneyu, normaltest, ranksums, chisquare
from statsmodels.stats.weightstats import zconfint, DescrStatsW, ztest

In [116]:
!pip install util-gfsilveira

In [117]:
from util import *

In [118]:
meus_uteis()

-------------
-=< Lista >=-
-------------
0 -> printLinha()
1 -> printLis()
2 -> printDic()
3 -> timeProcess()
4 -> import_started()
5 -> img_save()
6 -> setup_img_save()
7 -> mask_corr_graphic()
8 -> structured_confusion()


In [119]:
os.chdir('dados')

In [120]:
perguntas = joblib.load('perguntas.gz')
df = joblib.load('df_mysql.gz')
df.head()

,Data,IP,TCLE,Língua,Figura,Dintinguíveis,Proeminentes,Descrição,Eficiente,Útil,Estética,Fonte,Compreensão,Interesse,Grau,Área,Origem
id,,,,,,,,,,,,,,,,,
1,2020-03-02 13:31:20,200.134.9.2,s,pt,Reovírus Aviário,2,1,Sim,1,5,2,Sim,1,1,Doutorado,Exatas,Pesquisador
2,2020-03-02 17:45:41,191.52.71.228,s,pt,Reovírus Aviário,3,3,Sim,2,2,4,Não,4,2,Doutorado,Biológicas,Designer
3,2020-03-02 18:32:50,189.14.35.136,s,pt,Células Tronco,4,3,Sim,3,5,4,Não,3,3,Doutorado,Exatas,Pesquisador
4,2020-03-02 19:33:38,170.239.53.11,s,pt,Células Tronco,4,4,Sim,5,5,5,Sim,5,5,Mestrado,Biológicas,Designer
5,2020-03-03 11:50:20,189.103.158.150,s,pt,T. gondii,5,1,Sim,5,5,2,Sim,5,4,Doutorado,Biológicas,Pesquisador


In [121]:
# df.to_csv('dados_completos.csv')

In [122]:
perguntas

{'Compreensão': '6) Classifique o quanto a imagem facilitou a compreensão do conteúdo da informação.',
 'Descrição': '8) Existem elementos da imagem que requerem descrição adicional?',
 'Dintinguíveis': '1) Na imagem, o quão claramente os elementos são distinguíveis?',
 'Eficiente': '3) O quão eficiente são os elementos da imagem na representação dos dados?',
 'Estética': '5) O quanto a imagem é esteticamente bem elaborada?',
 'Fonte': '9) A imagem apresentada faz você experimentar uma nova fonte de dados?',
 'Grau': 'Qual o seu grau de formação acadêmica?',
 'Interesse': '7) Classifique o quanto a imagem despertou o seu interesse pelo conteúdo.',
 'Proeminentes': '2) O quão visualmente proeminentes são os elementos da imagem?',
 'Área': 'Área de formação?',
 'Útil': '4) O quão útil é a aplicação de uma imagem esquemática – desenho científico ou graphic abstract para representar os dados?'}

# Geral

In [123]:
df.head()

,Data,IP,TCLE,Língua,Figura,Dintinguíveis,Proeminentes,Descrição,Eficiente,Útil,Estética,Fonte,Compreensão,Interesse,Grau,Área,Origem
id,,,,,,,,,,,,,,,,,
1,2020-03-02 13:31:20,200.134.9.2,s,pt,Reovírus Aviário,2,1,Sim,1,5,2,Sim,1,1,Doutorado,Exatas,Pesquisador
2,2020-03-02 17:45:41,191.52.71.228,s,pt,Reovírus Aviário,3,3,Sim,2,2,4,Não,4,2,Doutorado,Biológicas,Designer
3,2020-03-02 18:32:50,189.14.35.136,s,pt,Células Tronco,4,3,Sim,3,5,4,Não,3,3,Doutorado,Exatas,Pesquisador
4,2020-03-02 19:33:38,170.239.53.11,s,pt,Células Tronco,4,4,Sim,5,5,5,Sim,5,5,Mestrado,Biológicas,Designer
5,2020-03-03 11:50:20,189.103.158.150,s,pt,T. gondii,5,1,Sim,5,5,2,Sim,5,4,Doutorado,Biológicas,Pesquisador


In [124]:
df.describe()

,Dintinguíveis,Proeminentes,Eficiente,Útil,Estética,Compreensão,Interesse
count,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000
mean,3.786017,3.576271,3.552966,4.614407,3.567797,3.599576,3.489407
std,1.227245,1.274654,1.247227,0.909751,1.340082,1.314075,1.376129
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,5.000000,3.000000,3.000000,2.000000
50%,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472 entries, 1 to 472
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Data           472 non-null    datetime64[ns]
 1   IP             472 non-null    object        
 2   TCLE           472 non-null    object        
 3   Língua         472 non-null    object        
 4   Figura         472 non-null    object        
 5   Dintinguíveis  472 non-null    int32         
 6   Proeminentes   472 non-null    int32         
 7   Descrição      472 non-null    object        
 8   Eficiente      472 non-null    int32         
 9   Útil           472 non-null    int32         
 10  Estética       472 non-null    int32         
 11  Fonte          472 non-null    object        
 12  Compreensão    472 non-null    int32         
 13  Interesse      472 non-null    int32         
 14  Grau           472 non-null    object        
 15  Área           472 non-

# Agrupando

## Analisando a pergunta Descrição

In [126]:
colunas_perguntas = df.columns[[7,11,14,15,16]]
df_quantitativo = df[colunas_perguntas]
df_quantitativo.head()

,Descrição,Fonte,Grau,Área,Origem
id,,,,,
1,Sim,Sim,Doutorado,Exatas,Pesquisador
2,Sim,Não,Doutorado,Biológicas,Designer
3,Sim,Não,Doutorado,Exatas,Pesquisador
4,Sim,Sim,Mestrado,Biológicas,Designer
5,Sim,Sim,Doutorado,Biológicas,Pesquisador


In [127]:
df_descricao = df_quantitativo[colunas_perguntas[[0,4]]].value_counts(normalize=False).reset_index()
df_descricao

,Descrição,Origem,0
0,Sim,Pesquisador,206
1,Sim,Designer,168
2,Não,Designer,68
3,Não,Pesquisador,30


In [128]:
dic_descricao = {
    df_descricao.iloc[0,1]: [df_descricao.iloc[0,2],df_descricao.iloc[3,2]],
    df_descricao.iloc[1,1]: [df_descricao.iloc[1,2],df_descricao.iloc[2,2]],
}
dic_descricao

{'Designer': [168, 68], 'Pesquisador': [206, 30]}

In [129]:
index=['Sim','Não']
df_descricao_compila = pd.DataFrame(data=np.nan, index=index, columns=dic_descricao.keys())
df_descricao_compila['Pesquisador'] = dic_descricao['Pesquisador']
df_descricao_compila['Designer'] = dic_descricao['Designer']
df_descricao_compila

,Pesquisador,Designer
Sim,206,168
Não,30,68


In [130]:
frequencias = [[df_descricao_compila.iloc[0,0],df_descricao_compila.iloc[1,0]],
              [df_descricao_compila.iloc[0,1],df_descricao_compila.iloc[1,1]]]
frequencias

[[206, 30], [168, 68]]

In [131]:
_, p_valor = chisquare(f_exp=df_descricao_compila['Pesquisador'], f_obs=df_descricao_compila['Designer'])
if p_valor < .001:
  p_valor = '<0.001'

df_descricao_compila['p-valor'] = (p_valor, np.nan)
df_descricao_compila.to_csv('df_descricao_compila.csv')
df_descricao_compila

,Pesquisador,Designer,p-valor
Sim,206,168,<0.001
Não,30,68,NaN


## Analisando a pergunta Fonte

In [132]:
colunas_perguntas = df.columns[[7,11,14,15,16]]
df_quantitativo = df[colunas_perguntas]
df_quantitativo.head()

,Descrição,Fonte,Grau,Área,Origem
id,,,,,
1,Sim,Sim,Doutorado,Exatas,Pesquisador
2,Sim,Não,Doutorado,Biológicas,Designer
3,Sim,Não,Doutorado,Exatas,Pesquisador
4,Sim,Sim,Mestrado,Biológicas,Designer
5,Sim,Sim,Doutorado,Biológicas,Pesquisador


In [133]:
df_fonte = df_quantitativo[colunas_perguntas[[1,4]]].value_counts(normalize=False).reset_index()
df_fonte

,Fonte,Origem,0
0,Sim,Designer,191
1,Sim,Pesquisador,146
2,Não,Pesquisador,90
3,Não,Designer,45


In [134]:
dic_fonte = {
    df_fonte.iloc[0,1]: [df_fonte.iloc[0,2],df_fonte.iloc[3,2]],
    df_fonte.iloc[1,1]: [df_fonte.iloc[1,2],df_fonte.iloc[2,2]],
}
dic_fonte

{'Designer': [191, 45], 'Pesquisador': [146, 90]}

In [135]:
index=['Sim','Não']
df_fonte_compila = pd.DataFrame(data=np.nan, index=index, columns=dic_fonte.keys())
df_fonte_compila['Pesquisador'] = dic_fonte['Pesquisador']
df_fonte_compila['Designer'] = dic_fonte['Designer']
df_fonte_compila = df_fonte_compila[df_fonte_compila.columns[[1,0]]]
df_fonte_compila

,Pesquisador,Designer
Sim,146,191
Não,90,45


In [136]:
frequencias = [[df_fonte_compila.iloc[0,0],df_fonte_compila.iloc[1,0]],
              [df_fonte_compila.iloc[0,1],df_fonte_compila.iloc[1,1]]]
frequencias

[[146, 90], [191, 45]]

In [137]:
_, p_valor = chisquare(f_exp=df_fonte_compila['Pesquisador'], f_obs=df_fonte_compila['Designer'])
if p_valor < .001:
  p_valor = '<0.001'

df_fonte_compila['p-valor'] = (p_valor, np.nan)
df_fonte_compila.to_csv('df_fonte_compila.csv')
df_fonte_compila

,Pesquisador,Designer,p-valor
Sim,146,191,<0.001
Não,90,45,NaN
